In [3]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
import csv
import random

dataset = "./datasets/iris.csv"
#desc: 4 input node, 3 output node, 5 hidden layer's node, 1 hidden layer, 20 first weights and 15 second weights
a = 0.5 #learning rate
w1 = np.random.rand(1,20)
w2 = np.random.rand(1,15)
print(w1)
print(w2)

[[0.74976805 0.68159843 0.8474241  0.10722035 0.55647558 0.99447804
  0.29268889 0.79366889 0.37987217 0.04161174 0.96321377 0.41394177
  0.26440519 0.58737711 0.89207147 0.29216026 0.53404336 0.90016904
  0.41709208 0.77073225]]
[[0.3226517  0.10743644 0.99039175 0.45502315 0.99568309 0.50128545
  0.34438633 0.64180363 0.88104774 0.90202462 0.55971401 0.43205403
  0.76684492 0.51379752 0.46425308]]
